# Import

In [76]:
import pandas as pd

In [77]:
import matplotlib.pyplot as plt

# 한글 폰트 지정
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

font_path = "C:/Windows/Fonts/malgun.ttf"

# Load Data

In [150]:
train_df = pd.read_csv("C:/Users/KimDongyoung/Desktop/GBT해커톤/GBT_hackerton/data/train.csv")
test_df = pd.read_csv("C:/Users/KimDongyoung/Desktop/GBT해커톤/GBT_hackerton/data/test.csv")  
submission = pd.read_csv("C:/Users/KimDongyoung/Desktop/GBT해커톤/GBT_hackerton/data/sample_submission.csv")  

In [151]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54609 entries, 0 to 54608
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      54609 non-null  object
 1   분류      54609 non-null  object
 2   제목      54609 non-null  object
 3   키워드     54609 non-null  object
dtypes: object(4)
memory usage: 1.7+ MB


In [152]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23405 entries, 0 to 23404
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      23405 non-null  object
 1   제목      23405 non-null  object
 2   키워드     23405 non-null  object
dtypes: object(3)
memory usage: 548.7+ KB


In [153]:
submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23405 entries, 0 to 23404
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      23405 non-null  object
 1   분류      23405 non-null  object
dtypes: object(2)
memory usage: 365.8+ KB


In [154]:
len(train_df['분류'].value_counts())

56

타겟변수 -> '분류'이고 종류 56가지

In [155]:
test_df.head()

,ID,제목,키워드
0,TEST_00000,[부고] 김태수씨 별세 외,"김태수,별세,김태수씨,서울,광남초등학,교장,별세,김윤정,이노코리아,대표,희정,한성대..."
1,TEST_00001,"신규 확진 나흘째 세자릿수... 방역당국, 핼러윈 풍선효과 차단 총력","신규,확진,나흘,세자릿수,방역당국,핼러윈,풍선,효과,차단,총력,감염증,신종,코로나바..."
2,TEST_00002,"[서경이 만난 사람] 전해철 장관 ""재정분권 강화 '지방자치 2.0 시대' 마중물 ...","전해철,장관,재정,분권,강화,지방자치,2.0,시대,마중물,마련,장관,전해철,행정안전..."
3,TEST_00003,"용인시, 12일 '장애인 구인 구직 만남의 날' 채용 행사","용인시,구인,장애인,구직,만남,채용,행사,노호근,용인특례시,장애인,취업,지원,대회의..."
4,TEST_00004,지자체 벽 터 경기지역 산단 활성화 모색,"지자체,경기,북동부,지역,산업단지,혁신단위,설정,전략,지역,연계,특성,제시,경기도경..."


영어 번역 내용 제거

In [157]:
# '키워드' 열에서 '기사'와 '구글' 두 단어를 모두 포함하는 행을 찾는 조건
condition = train_df['키워드'].apply(lambda x: all(word in x for word in ['기사', '구글']))

# 조건을 만족하는 행들로 새로운 데이터프레임 생성
filtered_df = train_df[condition].copy()

# '기사, 구글' 순서로 단어가 나오는 부분을 찾아 삭제하는 함수 정의
def remove_keywords(text):
    keywords = ['기사', '구글']
    start_index = 0
    for keyword in keywords:
        start_index = text.find(keyword, start_index)
        if start_index == -1:
            return text
        start_index += len(keyword)
    return text[:text.find('기사')].strip()

# 각 행의 '키워드' 값에서 '기사' 단어부터 마지막 단어까지 삭제
filtered_df['키워드'] = filtered_df['키워드'].apply(remove_keywords)

# 결과 확인
filtered_df.info()
print(filtered_df)

<class 'pandas.core.frame.DataFrame'>
Index: 2249 entries, 38 to 54576
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      2249 non-null   object
 1   분류      2249 non-null   object
 2   제목      2249 non-null   object
 3   키워드     2249 non-null   object
dtypes: object(4)
memory usage: 87.9+ KB
                ID       분류                                       제목  \
38     TRAIN_00038  정치:정치일반        이상일 용인특례시장, 민주평통 경기지역 의장표창 수여식 참석   
75     TRAIN_00075       지역             용인특례시, 식품산업 박람회 열어 우수 농산물 홍보   
86     TRAIN_00086       지역               용인특례시, 경기도 특별조정교부금 70억원 확보   
109    TRAIN_00109       지역       용인특례시, 모현 공동주택 현장 ‘찾아가는 행복소통시장실’가동   
137    TRAIN_00137    경제:유통          용인특례시, ‘백옥쌀’ 활용한 제빵용 ‘백옥쌀가루’ 출시   
...            ...      ...                                      ...   
54473  TRAIN_54473       지역      용인특례시, 서울3호선 연장...수원,성남,화성시 경기도와 맞손   
54481  TRAIN_54481       지역                        용인시, 생활폐기물 감량

In [159]:
filtered_df.iloc[157]['키워드']

'용인시,지반조사보고서,대상,기준,마련,용인시,기흥구,의무적,건축물,착공,의무,제출,지반조사보고서,제출,대상,기준,마련,지반조사보고서,건축물,설계,토지,형질,지반,내력,지반,조사,보고서,관련법,건축물,착공,제출,허가권자,인정,제출,예외,규정,해석,현장,혼란,가중,완화,지반,조사,보고서,제출,대상,지침,주변,건축물,지반,조사,결과,적용,별도,지반조사,건축물,구조기준,규칙,건축물,지반,최저,등급,가정,지반,규정,가지,예외,기준,마련,기준,대지,인접,대지,지반,조사,보고서,결과,지반,S4,S5,가정,구조안전확인서,제출,증축,수직,대수선,필로티,증축,구조기술사,건축사,별도,지반조사,인정,내년,건축물,적용,계획,관계자,지반,조사,보고서,제출,대상,기준,마련,일관성,건축,행정,추진,시민,만족,민원,처리,노력,'

In [160]:
len(filtered_df['분류'].value_counts())

47

In [161]:
filtered_df['분류'].value_counts()

분류
지역               1594
IT_과학:인터넷_SNS      59
사회:장애인             38
사회:사회일반            36
사회:교육_시험           35
경제:취업_창업           34
경제:반도체             34
문화:전시_공연           30
경제:부동산             24
스포츠:올림픽_아시안게임      22
정치:행정_자치           22
경제:경제일반            21
IT_과학:콘텐츠          20
사회:의료_건강           20
사회:여성              18
경제:자동차             18
정치:국회_정당           18
경제:무역              17
사회:환경              16
정치:정치일반            15
문화:문화일반            14
경제:유통              13
IT_과학:모바일          12
경제:금융_재테크          11
문화:출판              11
문화:학술_문화재          10
경제:서비스_쇼핑          10
문화:방송_연예            9
문화:미술_건축            9
스포츠:축구              9
정치:선거               7
문화:음악               5
사회:노동_복지            5
정치:청와대              4
IT_과학:보안            4
IT_과학:IT_과학일반       4
스포츠:월드컵             3
사회:사건_사고            3
국제                  3
경제:자원               3
문화:종교               2
정치:북한               2
IT_과학:과학            1
사회:미디어              1
문화:요리_여행            1
경제:산업_기

In [88]:
filtered_df.iloc[0]['키워드']

'이상일,용인,특례,시장,참석,민주평통,경기,지역,의장,표창,수여식,시장,민주평통,평화,수호,선도,활동,감사,강조,이상일,용인,특례,시장,에이스홀,시청,민주평화통일자문회,수여식,경기,지역,의장,표창,참석,관계자들,격려,자리,시장,홍승표,민주평통,경기,지역,회의,부의장,용인시협의회장,추상구,민주평통,용인시,협의회장,자문위원,민주평통,경기,지역,자문,위원,400여명,참석,시장,민주평화통일자문회,경기지역회의,안보태세,수호,한반도,평화,선도,활동,감사,시장,북한,도발,주시,차례,특강,정세,한반도,안보,강화,강조,내년,민주평통,활동,응원,차원,지원,수여식,이날,표창,임명숙,자문위원,장석순,용인시협의회,자문,위원,자치분권,정지혜,용인시,자치,분권,민간협력팀장,62명,유공자,의장,표창,용인시협의회,평화통일포럼,통일공감사업,진행,홍보,평화,통일,정책,노력,유공,기관,표창,대표,이소연,뉴코리아여성연합,통일,페이스,주제,특강,진행,'

실제 제거하여 데이터셋에 반영

In [162]:
# '키워드' 열에서 '기사'와 '구글' 두 단어를 모두 포함하는 행을 찾는 조건
condition_train = train_df['키워드'].apply(lambda x: all(word in x for word in ['기사', '구글']))
condition_test = test_df['키워드'].apply(lambda x: all(word in x for word in ['기사', '구글']))

# 조건을 만족하는 행들로 새로운 데이터프레임 생성
filtered_train_df = train_df[condition_train].copy()
filtered_test_df = test_df[condition_test].copy()

# '기사, 구글' 순서로 단어가 나오는 부분을 찾아 삭제하는 함수 정의
def remove_keywords(text):
    keywords = ['기사', '구글']
    start_index = 0
    for keyword in keywords:
        start_index = text.find(keyword, start_index)
        if start_index == -1:
            return text
        start_index += len(keyword)
    return text[:text.find('기사')].strip()

# 각 행의 '키워드' 값에서 '기사' 단어부터 마지막 단어까지 삭제
filtered_train_df['키워드'] = filtered_train_df['키워드'].apply(remove_keywords)
filtered_test_df['키워드'] = filtered_test_df['키워드'].apply(remove_keywords)

# 수정된 '키워드' 값을 원래 데이터프레임에 반영
train_df.loc[condition_train, '키워드'] = filtered_train_df['키워드']
test_df.loc[condition_test, '키워드'] = filtered_test_df['키워드']

In [90]:
# # 수정된 '키워드' 값을 원래 데이터프레임에 반영
# train_df.loc[condition_train, '키워드'] = filtered_train_df['키워드']
# test_df.loc[condition_test, '키워드'] = filtered_test_df['키워드']

# # 수정된 데이터프레임을 CSV 파일로 저장
# train_df.to_csv('train_df_translate_del.csv', index=False, encoding='utf-8-sig')
# test_df.to_csv('test_df_translate_del.csv', index=False, encoding='utf-8-sig')

## 전처리

In [163]:
import re
from collections import Counter

# 불용어 제거
removed_keywords = []

def remove_invalid_keywords(keywords):
    pattern = re.compile(
        r'[A-Za-z]+[가-힣\u4E00-\u9FFF]+|'  # 영어+한글
        r'[0-9]+[가-힣\u4E00-\u9FFF]+|'     # 숫자+한글
        r'[0-9]+[A-Za-z]+|'                 # 숫자+영어
        r'[가-힣]+[A-Za-z\u4E00-\u9FFF]+|'  # 한글+영어
        r'[가-힣]+[0-9]+|'                  # 한글+숫자
        r'[A-Za-z]+[0-9]+|'                 # 영어+숫자
        r'[\u4E00-\u9FFF]+|'                # 한자
        r'[0-9]+(\.[0-9]+)?%|'              # 숫자+퍼센트
        r'[0-9]+|'                          # 숫자
        r'[A-Za-z]+'                        # 영어
    )
    valid_keywords = []
    for word in keywords.split(','):
        word = word.strip()
        if not word or pattern.match(word):  # 공백이거나 패턴에 맞는 단어 제거
            removed_keywords.append(word)
        else:
            valid_keywords.append(word)
    return ', '.join(valid_keywords)

# 원본 데이터에서 해당 키워드들을 제거
train_df['키워드'] = train_df['키워드'].apply(remove_invalid_keywords)
test_df['키워드'] = test_df['키워드'].apply(remove_invalid_keywords)

# 결과 출력
print("\n수정된 데이터프레임:")
print(train_df)

# 제거된 단어들 출력
print("\n제거된 단어들:")
print(removed_keywords[:10])


수정된 데이터프레임:
                ID        분류                                     제목  \
0      TRAIN_00000  문화:전시_공연  용인문화재단, 인문학 콘서트 ‘당신이 모르는 뮤지컬 이야기Ⅳ’ 개최   
1      TRAIN_00001        지역           용인 농촌테마파크, 7~8월 단체체험객 체험료 지원   
2      TRAIN_00002        지역        용인시, 노후주택 에너지 성능 개선 신청 18일까지 연장   
3      TRAIN_00003        지역        수원 용인 고양시,‘특례시’로 지정 도시경쟁력 증가 기대   
4      TRAIN_00004        국제      용인시, 스페인 미국 국제명예자문관 위촉 대외홍보 지원 역할   
...            ...       ...                                    ...   
54604  TRAIN_54604        국제  용인 아파트서 30대 여성, 아들 딸 함께 추락 "극단 선택 추정"   
54605  TRAIN_54605  사회:교육_시험                    용인시, '위탁부모 보수교육' 실시   
54606  TRAIN_54606        지역          용인시, '플랫폼 시티' 국토부에 사업인정 협의 신청   
54607  TRAIN_54607        지역      용인시 이동읍 주민자치위원회, 저소득 20가구에 밑반찬 지원   
54608  TRAIN_54608        지역                 용인시-용인시공무원노조, 국무총리상 수상   

                                                     키워드  
0      용인문화재단, 인문학, 콘서트, 뮤지컬, 이야기, 개최, 인문학, 콘서트, 뮤지컬,...  
1      용인, 농촌, 테

In [164]:
# 제거된 단어들의 빈도 계산 및 상위 50개 출력
keyword_counter = Counter(removed_keywords)
top_50_keywords = keyword_counter.most_common(50)

print("\n제거된 단어들 상위 50개:")
for keyword, count in top_50_keywords:
    print(f"{keyword}: {count}")


제거된 단어들 상위 50개:
코로나19: 20588
A씨: 13186
: 7207
1만: 6451
SK하이닉스: 5605
AI: 5186
1년: 4812
SK: 4704
GTX: 4231
LH: 4175
3년: 4033
2년: 3948
B씨: 3770
6월: 3513
1명: 3455
12월: 3436
7월: 3407
3일: 3389
3월: 3347
1억: 3274
1천: 3178
2만: 3122
9월: 3099
4월: 3056
2명: 3024
3만: 2953
10월: 2874
5월: 2849
8월: 2814
1: 2791
3명: 2749
5년: 2740
7일: 2740
22일: 2734
23일: 2725
27일: 2708
8일: 2693
14일: 2684
11월: 2684
30일: 2668
28일: 2663
13일: 2641
IC: 2620
15일: 2606
20일: 2600
1일: 2590
26일: 2579
18일: 2577
3개: 2561
4일: 2510


In [171]:
# 정확히 일치하는 행 필터링
filtered_df = train_df[train_df['키워드'].str.contains(r'번역', na=False)]

# 결과 출력
filtered_df.iloc[3]['제목']

'용인특례시, 속초시와 자매 결연 주민복지 긴밀 협력'

In [172]:
# 정확히 일치하는 행 필터링
filtered_df = train_df[train_df['키워드'].str.contains(r'번역', na=False)]

# 결과 출력
filtered_df.iloc[3]['키워드']

'용인특례시, 속초시, 자매, 결연, 주민복지, 협력, 함평군, 전남, 경기도, 용인, 특례, 강원도, 속초시, 결연, 자매, 협약, 도시, 주민, 복지, 향상, 협력관계, 유지, 우수시책, 주민, 자치, 우수, 사례, 공유, 협력, 적극, 용인시민, 속초, 해수욕장, 공영, 주차장, 무료, 이용, 속초시립박물관, 입장료, 속초시민, 용인자연휴양림, 농촌, 테마파크, 입장료, 면제, 도시, 대표, 축제, 행사, 성공, 개최, 적극, 홍보, 예술인, 활동, 무대, 용인특례시립청소년오케스트라, 속초시립합창단, 문화예술단체, 교류, 문화, 예술, 단체, 활성화, 지역, 경제, 활성, 용인시, 토요일, 기흥역, 직거래장터, 속초시, 특산품, 판매, 속초시, 수출기업, 서포터즈, 용인, 시민, 통번역, 서포, 터즈, 지원, 이상일, 용인, 특례, 시장, 용인특례시, 속초시, 공동, 발전, 자매결연, 생각, 도시, 교류, 활성화, 시민들, 교류, 촉진, 상호, 발전, 협력, 도시, 농수산물, 직거래, 활성화, 관광, 교류, 활성, 시책, 교환, 활동, 도시, 발전, 도시, 시민, 행복, 증진, 이병선, 속초, 시장, 전국, 주목, 용인특례시, 자매도시, 결연, 생각, 대한민국, 경제, 중심축, 미래, 먹거리, 대표, 도시, 용인특례시, 우애, 협력, 도시, 최선, 용인시, 자매결연, 함평군, 전남, 충북, 단양군, 전남, 광양시, 전국, 우수, 도시, 결연, 발전, 지역, 균형, 노력, 방침'

In [173]:
# 제거된 단어들의 빈도 계산 및 상위 10개 출력
keyword_counter = Counter(removed_keywords)
top_10_keywords = keyword_counter.most_common(10)

print("\n제거된 단어들 상위 10개:")
for keyword, count in top_10_keywords:
    print(f"{keyword}: {count}")


제거된 단어들 상위 10개:
코로나19: 20588
A씨: 13186
: 7207
1만: 6451
SK하이닉스: 5605
AI: 5186
1년: 4812
SK: 4704
GTX: 4231
LH: 4175


In [174]:
train_df
# test_df


,ID,분류,제목,키워드
0,TRAIN_00000,문화:전시_공연,"용인문화재단, 인문학 콘서트 ‘당신이 모르는 뮤지컬 이야기Ⅳ’ 개최","용인문화재단, 인문학, 콘서트, 뮤지컬, 이야기, 개최, 인문학, 콘서트, 뮤지컬,..."
1,TRAIN_00001,지역,"용인 농촌테마파크, 7~8월 단체체험객 체험료 지원","용인, 농촌, 테마파크, 단체, 체험객, 체험료, 지원, 체험일, 기준, 용인시통합..."
2,TRAIN_00002,지역,"용인시, 노후주택 에너지 성능 개선 신청 18일까지 연장","용인시, 노후, 주택, 에너지, 성능, 개선, 신청, 연장, 용인시청, 용인시, 노..."
3,TRAIN_00003,지역,"수원 용인 고양시,‘특례시’로 지정 도시경쟁력 증가 기대","수원, 용인, 고양시, 특례시, 지정, 도시경쟁력, 증가, 경기, 도내, 인구, 수..."
4,TRAIN_00004,국제,"용인시, 스페인 미국 국제명예자문관 위촉 대외홍보 지원 역할","용인시, 스페인, 미국, 국제, 명예, 자문관, 위촉, 역할, 대외, 홍보, 지원,..."
...,...,...,...,...
54604,TRAIN_54604,국제,"용인 아파트서 30대 여성, 아들 딸 함께 추락 ""극단 선택 추정""","용인, 아파트, 여성, 아들, 추락, 극단, 선택, 추정, 경찰, 현장, 유서, 경..."
54605,TRAIN_54605,사회:교육_시험,"용인시, '위탁부모 보수교육' 실시","용인시, 위탁, 부모, 보수, 교육, 용인시, 경기, 남부, 가정, 위탁, 지원, ..."
54606,TRAIN_54606,지역,"용인시, '플랫폼 시티' 국토부에 사업인정 협의 신청","용인시, 플랫폼, 시티, 국토부, 신청, 사업, 인정, 협의, 경기, 용인시, 중앙..."
54607,TRAIN_54607,지역,"용인시 이동읍 주민자치위원회, 저소득 20가구에 밑반찬 지원","주민자치위원회, 용인시, 이동읍, 주민, 자치, 위원회, 가구, 밑반찬, 지원, 용..."


In [176]:
# 인덱스 5번의 키워드 컬럼 전체를 출력
index = 15489
if index in train_df.index:
    print(train_df.loc[index, '키워드'])
else:
    print(f"인덱스 {index}가 데이터프레임에 존재하지 않습니다.")

매출, 쓱세일, 대박, 용진이, 이마트, 노조, 용진이, 사원들, 쓱세일, 기간, 사흘, 사흘, 매출, 노조, 요구, 이마트, 결정, 신세계그룹, 진행, 쓱세일, 성황리, 마무리, 시장, 여운, 전국이마트노동조합, 한국노총, 소속, 쓱세일, 매출, 보상, 요구, 부회장, 정용진, 신세계그룹, 이목, 점포, 오픈런, 임시휴점, 정도, 사람, 기간, 이마트, 매출, 관심, 신선식품, 공산품, 쓱세일, 흥행, 행사, 신세계그룹, 계열사, 야구단, 랜더스, 우승, 리그, 통합, 기념, 개최, 이마트, 인기, 카테고리, 원플러스원, 혜택, 할인, 최대, 적용, 매장, 카트, 정도, 흥행, 성공, 이마트, 쓱세일, 매출, 쓱세일, 원래, 예상, 수치, 이마트, 노조, 용진이, 사원들, 제목, 성명서, 보상, 요구, 이마트, 노조, 쓱세일, 이마트, 사원, 고객, 안전, 상품, 진열, 응대, 눈코, 그룹, 지탱, 이마트, 사원, 보상, 이마트, 노조, 노사, 임금, 협상, 교섭, 우승, 랜더스, 시리즈, 기념, 사원, 지급, 요구, 노조, 이마트, 이마트, 시작, 사원들, 피땀, 수익, 캐시카우, 역할, 강조, 이마트, 행사, 기간, 매출, 전년, 동기, 대비, 물량, 준비, 삼겹살, 목살, 할인, 적용, 사흘, 매출, 사흘, 기록, 투플러스원, 적용, 봉지, 적용, 참치, 통조림, 매출, 증가, 계란, 상품, 이판란, 완판, 계란, 매출, 계란, 경기, 용인시, 이마트, 주부, 서모, 사람들, 인산인해, 인기, 품목, 매대, 마트, 신선식품, 공산품, 코너, 모습, 이마트, 행사, 기간, 세제, 치약, 샴푸, 생활용품, 매출, 상승, 이마트, 할인, 행사, 진행, 행사, 규모, 연중, 최대, 할인, 사람들, 우승, 야구, 마케팅, 집객, 효과, 이마트, 관계자, 물가, 장기화, 상황, 장바구니, 안정, 행사, 진행, 고객, 시선, 매출, 미소, 이마트, 노조, 요구, 미지수, 이마트, 관계자, 이마트, 노조, 요구, 내부, 협의, 결정


In [177]:
combined_df = pd.concat([train_df, test_df], ignore_index=True)

In [178]:
def extract_single_character_keywords(df, keyword_column):
    """
    데이터프레임의 각 행에서 한 글자인 키워드를 추출합니다.
    """
    single_char_keywords = []
    
    for index, row in df.iterrows():
        if pd.notna(row[keyword_column]):
            keywords_list = row[keyword_column].split(',')
            single_char_keywords.extend([keyword for keyword in keywords_list if len(keyword) == 1])
    
    return single_char_keywords

def count_single_character_keywords(df, keyword_column):
    """
    데이터프레임의 각 행에서 한 글자인 키워드를 추출하고, 빈도를 계산하여 딕셔너리 형태로 반환합니다.
    """
    single_char_keywords = extract_single_character_keywords(df, keyword_column)
    keyword_counts = Counter(single_char_keywords)
    return dict(keyword_counts)

single_char_keyword_counts = count_single_character_keywords(combined_df, '키워드')
print(single_char_keyword_counts)

{}


In [179]:
def extract_top_keywords_by_group(df, group_column, keyword_column, top_n=5):
    # 그룹화
    grouped = df.groupby(group_column)
    
    top_keywords = {}
    
    for group_name, group_df in grouped:
        # 모든 키워드 결합
        all_keywords = group_df[keyword_column].dropna().str.cat(sep=',')
        
        # 키워드 분리 및 빈도 계산
        keywords_list = all_keywords.split(',')
        keyword_counts = Counter(keywords_list)
        
        # 가장 많이 등장하는 키워드 추출
        if keyword_counts:
            top_keywords[group_name] = [keyword for keyword, count in keyword_counts.most_common(top_n)]
        
        # 각 그룹의 키워드를 줄바꿈으로 구분하여 출력
        print(f"Group: {group_name}, Keywords: {top_keywords[group_name]}\n")
    
    return top_keywords

# 사용 예시
top_keywords = extract_top_keywords_by_group(train_df, '분류', '키워드', top_n=5)
print(top_keywords)

Group: IT_과학:IT_과학일반, Keywords: [' 용인시', ' 캐릭터', ' 데이터', ' 센터', ' 교육']

Group: IT_과학:과학, Keywords: [' 로봇', ' 기업', ' 임상', ' 용인시', ' 진행']

Group: IT_과학:모바일, Keywords: [' 서비스', ' 용인시', ' 도서관', ' 제공', ' 이용']

Group: IT_과학:보안, Keywords: [' 정보', ' 용인시', ' 해킹', ' 개인', ' 아파트']

Group: IT_과학:인터넷_SNS, Keywords: [' 용인시', ' 온라인', ' 홈페이지', ' 서비스', ' 교육']

Group: IT_과학:콘텐츠, Keywords: [' 서비스', ' 콘텐츠', ' 용인시', ' 교육', ' 운영']

Group: 경제:경제일반, Keywords: [' 용인시', ' 용인', ' 시장', ' 경기', ' 지역']

Group: 경제:금융_재테크, Keywords: [' 은행', ' 지원', ' 대출', ' 점포', ' 금융']

Group: 경제:무역, Keywords: [' 수출', ' 지원', ' 기업', ' 시장', ' 달러']

Group: 경제:반도체, Keywords: [' 반도체', ' 용인', ' 산업', ' 기업', ' 조성']

Group: 경제:부동산, Keywords: [' 아파트', ' 분양', ' 단지', ' 가구', ' 서울']

Group: 경제:산업_기업, Keywords: [' 회장', ' 기업', ' 경영', ' 용인시', ' 사업']

Group: 경제:서비스_쇼핑, Keywords: [' 용인시', ' 고객', ' 브랜드', ' 매장', ' 지역']

Group: 경제:외환, Keywords: [' 정부', ' 인상', ' 인하', ' 금리', ' 가격']

Group: 경제:유통, Keywords: [' 가격', ' 용인시', ' 지역', ' 상품', ' 시장']

Group: 경제:자동차, K

In [180]:
# 용인시, 용인 제거

def remove_specific_keywords(df, keyword_column, keywords_to_remove):
    """
    데이터프레임의 각 행에서 특정 키워드를 제거합니다.
    """
    for index, row in df.iterrows():
        if pd.notna(row[keyword_column]):
            keywords_list = row[keyword_column].split(',')
            filtered_keywords = [keyword for keyword in keywords_list if keyword not in keywords_to_remove]
            df.at[index, keyword_column] = ','.join(filtered_keywords)
    
    return df

In [181]:
train_df = remove_specific_keywords(train_df, '키워드', [' 용인시', ' 용인'])
test_df = remove_specific_keywords(test_df, '키워드', [' 용인시', ' 용인'])

In [182]:
top_keywords = extract_top_keywords_by_group(train_df, '분류', '키워드', top_n=10)
print(top_keywords)

Group: IT_과학:IT_과학일반, Keywords: [' 캐릭터', ' 데이터', ' 센터', ' 교육', ' 조아용', ' 지역', ' 서비스', ' 활용', ' 분야', ' 사업']

Group: IT_과학:과학, Keywords: [' 로봇', ' 기업', ' 임상', ' 진행', ' 활용', ' 사업', ' 대표', ' 혁신', ' 분야', ' 뇌졸중']

Group: IT_과학:모바일, Keywords: [' 서비스', ' 도서관', ' 제공', ' 이용', ' 운영', ' 지원', ' 지역', ' 기업', ' 활용', ' 시장']

Group: IT_과학:보안, Keywords: [' 정보', ' 해킹', ' 개인', ' 아파트', ' 보호', ' 월패드', ' 관리', ' 방지', ' 제공', ' 구축']

Group: IT_과학:인터넷_SNS, Keywords: [' 온라인', ' 홈페이지', ' 서비스', ' 교육', ' 운영', ' 아동', ' 지원', ' 조아', ' 조아용', ' 시민들']

Group: IT_과학:콘텐츠, Keywords: [' 서비스', ' 콘텐츠', ' 교육', ' 운영', ' 시민들', ' 미디어', ' 제공', ' 활용', ' 진행', ' 시장']

Group: 경제:경제일반, Keywords: [' 시장', ' 경기', ' 지역', ' 서비스', ' 사업', ' 반도체', ' 지원', ' 운영', ' 서울', ' 시민']

Group: 경제:금융_재테크, Keywords: [' 은행', ' 지원', ' 대출', ' 점포', ' 금융', ' 운영', ' 지역', ' 보험', ' 우리은행', ' 하나은행']

Group: 경제:무역, Keywords: [' 수출', ' 지원', ' 기업', ' 시장', ' 달러', ' 중소기업', ' 해외', ' 사업', ' 전시회', ' 상담']

Group: 경제:반도체, Keywords: [' 반도체', ' 산업', ' 기업', ' 조성', ' 시장', ' 클러스터', '

In [103]:
# def remove_single_occurrence_keywords_per_row(df, keyword_column):
#     """
#     데이터프레임의 각 행에서 1번만 쓰인 단어를 제거합니다.
#     """
#     for index, row in df.iterrows():
#         if pd.notna(row[keyword_column]):
#             keywords_list = row[keyword_column].split(',')
#             keyword_counts = Counter(keywords_list)
#             filtered_keywords = [keyword for keyword in keywords_list if keyword_counts[keyword] > 1]
#             df.at[index, keyword_column] = ','.join(filtered_keywords)
    
#     return df

In [104]:
# train_df = remove_single_occurrence_keywords_per_row(train_df, '키워드')
# test_df = remove_single_occurrence_keywords_per_row(test_df, '키워드')

In [105]:
# # Get the keywords from the first row
# first_row_keywords = test_df.iloc[56]['키워드']
# # BEGIN: Find and count repeated keywords

# # Split the keywords and count occurrences
# keywords_list = first_row_keywords.split(',')
# keyword_counts = Counter(keywords_list)

# # Find repeated keywords
# repeated_keywords = {keyword: count for keyword, count in keyword_counts.items() if count == 1} # 각 행에서 키워드가 1번만 나온 키워드를 찾습니다.
# # END:

# repeated_keywords

In [106]:
# train_df.info()

In [108]:
def extract_rows_with_keyword(df, keyword_column, keyword_to_find):
    """
    데이터프레임의 각 행에서 특정 키워드가 포함된 행을 추출합니다.
    """
    df = df[df[keyword_column].str.contains(keyword_to_find, na=False)]
    return df

# 사용 예시
#d_f = pd.DataFrame({'키워드': ['사과,바나나,조아용,11명', '오렌지,포도,용인,3사람', '딸기,키위,abc123,한글영어123']})
keyword_to_find = ' 조아'
df = extract_rows_with_keyword(train_df, '키워드', keyword_to_find)

df

,ID,분류,제목,키워드
18,TRAIN_00018,경제:서비스_쇼핑,용인특례시 지역 우수농산물과 가공식품 추석맞이 판매,"우수농산물, 특례시, 지역, 우수, 농산물, 판매, 가공, 식품, 추석, 맞이, 로..."
19,TRAIN_00019,문화:전시_공연,"용인시 어린이날 축제장 찾은 아이들 ""너무 조아용"" 한목소리","축제장, 어린이날, 아이들, 조아용, 한목소리, 엄마, 조아용, 조아용, 아빠, 키..."
67,TRAIN_00067,지역,여권 간편발급 교복지원비 인상...용인시 104개 '시민 만족 서비스' 제공,"간편, 발급, 교복지원비, 인상, 시민, 만족, 서비스, 제공, 발급, 간편, 장애..."
121,TRAIN_00121,경제:자원,"용인시, 관광형 DRT 브랜드명 '타바용'","용인시, 관광, 브랜드명, 타바용, 한국, 민속촌, 일대, 교통, 거점, 운행, 관..."
244,TRAIN_00244,지역,"상복 터진 용인,사람 중심 정책 통했다","상복, 사람, 중심, 정책, 지역, 사업, 세계, 인정, 도시, 각종, 쾌속, 질주..."
...,...,...,...,...
54245,TRAIN_54245,지역,"굿즈 '조아용' 인기몰이 용인시, 수익금 저소득층에 재투자","굿즈, 조아, 인기몰이, 수익금, 소득, 재투자, 고향, 캐릭터, 조아, 굿즈, 답..."
54259,TRAIN_54259,문화:전시_공연,"""용인시민들 모여용! 놀아용! 조아용"" 23~24일 '2023 용인시민 페스타' 개최","용인시민들, 조아용, 페스타, 시민, 개최, 용인특례시, 용인미르스타디움, 시민, ..."
54318,TRAIN_54318,IT_과학:IT_과학일반,"에버랜드, '레시 & 조아용' 캐릭터 굿즈 인기폭발","에버랜드, 레시, 조아용, 인기폭발, 캐릭터, 굿즈, 인기, 폭발, 청룡, 용인특례..."
54426,TRAIN_54426,지역,MZ 초딩 사이에 인기몰이하는 ‘조아용’ 시청에 문 연 홍보관 발길 잇따라,"인기몰이, 조아용, 시청, 홍보관, 발길, 민원창구, 시청사, 민원, 창구, 조아,..."


In [109]:
df['분류'].value_counts()

분류
지역               324
문화:전시_공연          76
IT_과학:IT_과학일반     40
IT_과학:인터넷_SNS     39
경제:서비스_쇼핑         34
경제:유통             32
경제:반도체            25
스포츠:골프            22
경제:경제일반           19
문화:방송_연예          17
경제:자동차            16
IT_과학:콘텐츠         12
사회:교육_시험          12
IT_과학:모바일         10
문화:요리_여행          10
문화:문화일반            9
문화:생활              9
경제:자원              9
경제:취업_창업           9
정치:국회_정당           8
사회:사회일반            5
경제:산업_기업           4
사회:장애인             4
스포츠:올림픽_아시안게임      4
정치:정치일반            3
사회:환경              2
문화:미술_건축           2
사회:여성              2
경제:무역              2
문화:출판              2
국제                 1
문화:종교              1
사회:사건_사고           1
스포츠:스포츠일반          1
문화:영화              1
스포츠:월드컵            1
Name: count, dtype: int64

In [110]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23405 entries, 0 to 23404
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      23405 non-null  object
 1   제목      23405 non-null  object
 2   키워드     23405 non-null  object
dtypes: object(3)
memory usage: 548.7+ KB


In [183]:
test_df.to_csv('C:/Users/KimDongyoung/Desktop/GBT해커톤/GBT_hackerton/code/dongyoung/test_df_1002.csv')
train_df.to_csv('C:/Users/KimDongyoung/Desktop/GBT해커톤/GBT_hackerton/code/dongyoung/train_df_1002.csv')

In [184]:
test_df = pd.read_csv('C:/Users/KimDongyoung/Desktop/GBT해커톤/GBT_hackerton/code/dongyoung/test_df_1002.csv')

In [185]:
train_df.isna().sum()

ID     0
분류     0
제목     0
키워드    0
dtype: int64

In [186]:
test_df

,Unnamed: 0,ID,제목,키워드
0,0,TEST_00000,[부고] 김태수씨 별세 외,"김태수, 별세, 김태수씨, 서울, 광남초등학, 교장, 별세, 김윤정, 이노코리아, ..."
1,1,TEST_00001,"신규 확진 나흘째 세자릿수... 방역당국, 핼러윈 풍선효과 차단 총력","신규, 확진, 나흘, 세자릿수, 방역당국, 핼러윈, 풍선, 효과, 차단, 총력, 감..."
2,2,TEST_00002,"[서경이 만난 사람] 전해철 장관 ""재정분권 강화 '지방자치 2.0 시대' 마중물 ...","전해철, 장관, 재정, 분권, 강화, 지방자치, 시대, 마중물, 마련, 장관, 전해..."
3,3,TEST_00003,"용인시, 12일 '장애인 구인 구직 만남의 날' 채용 행사","용인시, 구인, 장애인, 구직, 만남, 채용, 행사, 노호근, 용인특례시, 장애인,..."
4,4,TEST_00004,지자체 벽 터 경기지역 산단 활성화 모색,"지자체, 경기, 북동부, 지역, 산업단지, 혁신단위, 설정, 전략, 지역, 연계, ..."
...,...,...,...,...
23400,23400,TEST_23400,코로나19 감염 경로 '조사중' 32.4% 최고치 일상감염 지속,"감염, 경로, 조사, 최고, 일상감염, 지속, 기준, 확진자, 기준, 신규, 집계,..."
23401,23401,TEST_23401,“여행 외식해라” vs “모임 자제하라” 시민들 “어쩌란 건가” 혼란,"여행, 외식, 자제, 모임, 시민들, 혼란, 인천국제공항, 아시아나항공, 한반도, ..."
23402,23402,TEST_23402,송철호 울산시장 배우자 용인 임야 쪼개기 매입 의혹,"임야, 송철호, 울산, 시장, 배우자, 매입, 의혹, 송철호, 울산, 시장, 배우자..."
23403,23403,TEST_23403,여직원 배에 '자궁 모형' 올리고 사진 찍어 홍보용으로 쓴 한의사,"여직원, 자궁, 모형, 사진, 홍보용, 한의사, 한의원, 간호조무사, 동의, 자궁,..."


In [189]:
def extract_first_n_keywords(df, keyword_column, n=20):
    """
    데이터프레임의 '키워드' 컬럼에서 각 행의 맨 처음 n개 단어를 추출합니다.
    """
    for index, row in df.iterrows():
        if pd.notna(row[keyword_column]):
            keywords_list = row[keyword_column].split(',')
            first_n_keywords = keywords_list[:n]
            df.at[index, keyword_column] = ','.join(first_n_keywords)
    
    return df

# 사용 예시
train_ = pd.DataFrame({
    '키워드': [
        '사과,바나나,용인시,11명,바나나,딸기,포도,오렌지,키위,망고,수박,참외,복숭아,자두,체리,블루베리,라즈베리,크랜베리,구아바,파파야,리치,두리안',
        '오렌지,포도,용인,3사람,사과,바나나,딸기,키위,abc123',
        '딸기,딸기,키위,abc123,한글영어123,사과,바나나,포도,오렌지,키위,망고,수박,참외,복숭아,자두,체리,블루베리,라즈베리,크랜베리,구아바,파파야,리치,두리안'
    ]
})
train_ = extract_first_n_keywords(train_, '키워드', 20)
train_

,키워드
0,"사과,바나나,용인시,11명,바나나,딸기,포도,오렌지,키위,망고,수박,참외,복숭아,자..."
1,"오렌지,포도,용인,3사람,사과,바나나,딸기,키위,abc123"
2,"딸기,딸기,키위,abc123,한글영어123,사과,바나나,포도,오렌지,키위,망고,수박..."


In [190]:
train_extrct_df = extract_first_n_keywords(train_df, '키워드', 20)

In [192]:
# 인덱스 5번의 키워드 컬럼 전체를 출력
index = 56
if index in train_extrct_df.index:
    print(train_extrct_df.loc[index, '키워드'])
else:
    print(f"인덱스 {index}가 데이터프레임에 존재하지 않습니다.")

정권, 이성, 집단, 안보, 윤석열, 대통령, 을지, 자유, 방패, 연습, 정권, 북한, 도발, 감행, 지구상, 비이성적, 집단, 안보, 태세, 오판
